# Modelling Bioethanol Production


This notebook shows the implementation of inhibition models for bioethanol production process from cassava. This example is availabe on Book from Schmidell et al (2001), entitled Industrial Biotechnology  (v2, 2001, p. 135, chap 7).

Experimental data presented in the book were extracted by using the tool [Webplot Digitalizer](https://automeris.io/WebPlotDigitizer/)

In [1]:
import numpy as np
from numpy import log as ln
from numpy import arange,array,ones 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from math import*
from sklearn.linear_model import LinearRegression
from scipy import interpolate
from scipy import stats
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/home/mone/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


## Open data from experiment

In [2]:
assay_1 = pd.read_csv('assay_1.csv', encoding='utf-8', decimal=',', sep=' ', header=None)
assay_2 = pd.read_csv('assay_2.csv', encoding='utf-8', decimal=',', sep=' ', header=None)
assay_3 = pd.read_csv('assay_3.csv', encoding='utf-8', decimal=',', sep=' ', header=None)
assay_4 = pd.read_csv('assay_4.csv', encoding='utf-8', decimal=',', sep=' ', header=None)

## Checking experimental data

In [3]:
assay_1

,0,1,2,3
0,0.0,0.378,1.92,20.800
1,1.0,0.652,2.54,17.600
2,2.0,1.170,3.54,14.800
3,3.0,1.540,4.65,10.300
4,4.0,1.840,5.96,5.800
5,5.0,2.360,6.64,2.340
6,6.0,2.200,7.19,0.512
7,7.0,2.230,6.74,0.088


In [4]:
assay_2

,0,1,2,3
0,0.0,0.845,2.44,85.100
1,1.0,1.080,2.88,76.800
2,2.0,1.880,3.54,76.300
3,3.0,2.980,5.34,74.800
4,4.0,3.920,7.52,56.900
5,5.0,5.770,10.50,42.200
6,6.0,7.140,17.60,28.800
7,7.0,10.600,22.80,7.650
8,8.0,10.300,24.70,0.198
9,9.0,7.700,24.40,0.002


In [5]:
assay_3

,0,1,2,3
0,0.0,0.410,2.71,136.000
1,1.0,0.819,2.78,130.000
2,2.0,1.140,3.06,131.000
3,3.0,1.720,3.43,134.000
4,4.0,2.570,4.78,130.000
5,5.0,4.010,6.78,120.000
6,6.0,4.680,8.34,106.000
7,7.0,6.600,11.70,100.000
8,8.0,9.510,15.40,69.800
9,9.0,12.600,23.00,47.500


In [6]:
assay_4

,0,1,2,3
0,0.0,1.12,2.02,227.000
1,1.0,1.29,2.56,236.000
2,2.0,2.29,2.90,221.000
3,3.0,2.68,3.82,213.000
4,4.0,4.36,4.44,198.000
5,5.0,6.18,6.69,198.000
6,6.0,7.70,9.31,195.000
7,7.0,11.10,11.30,178.000
8,8.0,13.60,15.20,160.000
9,9.0,18.30,21.00,123.000


## Transforming data in arrays

In [7]:
time_assay_1 = assay_1 [0]
biomass_assay_1 = assay_1[1]
product_assay_1 = assay_1[2]
substrate_assay_1 = assay_1[3]

In [8]:
time_assay_2 = assay_2 [0]
biomass_assay_2 = assay_2[1]
product_assay_2 = assay_2[2]
substrate_assay_2 = assay_2[3]

In [9]:
time_assay_3 = assay_3 [0]
biomass_assay_3 = assay_3[1]
product_assay_3 = assay_3[2]
substrate_assay_3 = assay_3[3]

In [10]:
time_assay_4 = assay_4 [0]
biomass_assay_4 = assay_4[1]
product_assay_4 = assay_4[2]
substrate_assay_4 = assay_4[3]

# Estimating specific growth rates (umax)

In [11]:
#assay 1: umax for times 2, 3 4 and 5 h of exponencial growth (Table 7.3, pg 140 )
ln_biomass_assay_1 = np.log(biomass_assay_1)
coef_a1_2h,coef_b1_2h = np.polyfit(time_assay_1[0:3], ln_biomass_assay_1[0:3], 1)
coef_a1_3h,coef_b1_3h = np.polyfit(time_assay_1[0:4], ln_biomass_assay_1[0:4], 1)
coef_a1_4h,coef_b1_4h = np.polyfit(time_assay_1[0:5], ln_biomass_assay_1[0:5], 1)
coef_a1_5h,coef_b1_5h = np.polyfit(time_assay_1[0:6], ln_biomass_assay_1[0:6], 1)
print("umax1 t=2h")
print (coef_a1_2h)
#print (coef_b1_2h)
print("umax1 t=3h")
print (coef_a1_3h)
#print (coef_b1_3h)
print("umax1 t=4h")
print (coef_a1_4h)
#print (coef_b1_4h)
print("umax1 t=5h")
print (coef_a1_5h)
#print (coef_b1_5h)

umax1 t=2h
0.564932416086107
umax1 t=3h
0.47986449652294094
umax1 t=4h
0.4024746443447911
umax1 t=5h
0.35842345844700985


In [12]:
#assay 2: umax for times 2h of exponencial growth 
ln_biomass_assay_2 = np.log(biomass_assay_2)
coef_a2_2h,coef_b2_2h = np.polyfit(time_assay_2[0:3], ln_biomass_assay_2[0:3], 1)
print("umax_2 t=2h")
print (coef_a2_2h)
#print (coef_b2_2h)

#assay 3: umax for times 2h of exponencial growth 
ln_biomass_assay_3 = np.log(biomass_assay_3)
coef_a3_2h,coef_b3_2h = np.polyfit(time_assay_3[0:3], ln_biomass_assay_3[0:3], 1)
print("umax_3 t=2h")
print (coef_a3_2h)
#print (coef_b3_2h)

#assay 4: umax for times 2h of exponencial growth 
ln_biomass_assay_4 = np.log(biomass_assay_4)
coef_a4_2h,coef_b4_2h = np.polyfit(time_assay_4[0:3], ln_biomass_assay_4[0:3], 1)
print("umax_4 t=2h")
print (coef_a4_2h)
#print (coef_b4_2h)

umax_2 t=2h
0.3998452142334105
umax_3 t=2h
0.5113131908450937
umax_4 t=2h
0.3576115661295725


In [13]:
#Estimation of uP for time 2h  during bacterial exponencial growth 

#assay 1:
ln_product_assay_1 = np.log(product_assay_1)
coef_a1p_2h,coef_b1p_2h = np.polyfit(time_assay_1[0:3], ln_product_assay_1[0:3], 1)
print("uP_1 t=2h")
print (coef_a2_2h)
#print (coef_b2p_2h)

#assay 2:
ln_product_assay_2 = np.log(product_assay_2)
coef_a2p_2h,coef_b2p_2h = np.polyfit(time_assay_2[0:3], ln_product_assay_2[0:3], 1)
print("uP_2 t=2h")
print (coef_a2p_2h)
#print (coef_b2p_2h)

#assay 3:
ln_product_assay_3 = np.log(product_assay_3)
coef_a3p_2h,coef_b3p_2h = np.polyfit(time_assay_3[0:3], ln_product_assay_3[0:3], 1)
print("uP_3 t=2h")
print (coef_a3p_2h)
#print (coef_b3p_2h)


#assay 4:
ln_product_assay_4 = np.log(product_assay_4)
coef_a4p_2h,coef_b4p_2h = np.polyfit(time_assay_4[0:3], ln_product_assay_4[0:3], 1)
print("uP_4 t=2h")
print (coef_a4p_2h)
#print (coef_b4p_2h)

uP_1 t=2h
0.3998452142334105
uP_2 t=2h
0.18606434392028637
uP_3 t=2h
0.06073314053633977
uP_4 t=2h
0.1808066127896575


In [14]:
#Initial Parameters to run model, considering umax for each timestep interval

#assay_1
ln_biomass_assay_1 = np.log(biomass_assay_1)
coef_a1_2hr,coef_b1_2h = np.polyfit(time_assay_1[0:3], ln_biomass_assay_1[0:3], 1)
coef_a1_3hr,coef_b1_3h = np.polyfit(time_assay_1[2:4], ln_biomass_assay_1[2:4], 1)
coef_a1_4hr,coef_b1_4h = np.polyfit(time_assay_1[3:5], ln_biomass_assay_1[3:5], 1)
coef_a1_5hr,coef_b1_5h = np.polyfit(time_assay_1[4:6], ln_biomass_assay_1[4:6], 1)
uxMax_assay_1 = [coef_a1_2hr, coef_a1_3hr, coef_a1_4hr, coef_a1_5hr]
initialCbio = 0.378 #g/L
dt = 1.0 #h

# Creating a function in order to check experimental values vs. those obtained by umax recovery

In [15]:
def computeParameters(cBio, timestep):   
  
    #log phase growth equation
    if timestep < 2.0:
        cBio = cBio * math.exp(uxMax_assay_1[0] * dt)
        return [cBio]
    if (timestep >= 2.0 and timestep < 3.0):
        cBio = cBio * math.exp(uxMax_assay_1[1] * dt)
        return [cBio]
    if timestep >= 3.0 and timestep < 4.0:
        cBio = cBio * math.exp(uxMax_assay_1[2] * dt)
        return [cBio]
    if timestep >= 4.0:
        cBio = cBio * math.exp(uxMax_assay_1[3] * dt)
        return [cBio]

In [16]:
#if __name__ == "__main__":
    timesteps = np.arange(0, 6.0, dt) # hrs
    biomassConcentration = np.array([initialCbio])

# loop collideAndStream
    count = 0
    for timestep in timesteps:
        [cBio] = computeParameters(biomassConcentration[-1], timestep)

        biomassConcentration = np.append(biomassConcentration, cBio) 
        count += 1

    biomassConcentration = np.delete(biomassConcentration, 0)

NameError: name 'math' is not defined

In [ ]:
biomassConcentration  

In [ ]:
biomass_assay_1

# Ploting data: Experimental vs. Estimated


In [ ]:
# This figure can be compared with fig. 7.3 from book (pg. 141)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(timesteps, biomassConcentration, 'g-')
ax2.plot(time_assay_1[0:7], biomass_assay_1[0:7], 'bo')
ax1.set_xlabel('time (h)')
ax1.set_ylabel('biomass (g/L) - Model data', color='g')
ax2.set_ylabel('biomass (g/L) - Experimental data', color='b')
ax2.set_ylim (0, 3)

# Recovering uX, uP and uS from experimental data : Geometric Method

In [ ]:
#uX

uX_assay_1 = np.diff(biomass_assay_1)/biomass_assay_1[1:]
print ('uX_assay_1')
print (uX_assay_1)

uX_assay_2 = np.diff(biomass_assay_2)/biomass_assay_2[1:]
print ('uX_assay_2')
print (uX_assay_2)

uX_assay_3 = np.diff(biomass_assay_3)/biomass_assay_3[1:]
print ('uX_assay_3')
print (uX_assay_3)

uX_assay_4 = np.diff(biomass_assay_4)/biomass_assay_4[1:]
print ('uX_assay_4')
print (uX_assay_4)

In [ ]:
#uP

uP_assay_1 = np.diff(product_assay_1)/product_assay_1[1:]
print ('uP_assay_1')
print (uP_assay_1)

uP_assay_2 = np.diff(product_assay_2)/product_assay_2[1:]
print ('uP_assay_2')
print (uP_assay_2)

uP_assay_3 = np.diff(product_assay_3)/product_assay_3[1:]
print ('uP_assay_3')
print (uP_assay_3)

uP_assay_4 = np.diff(product_assay_4)/product_assay_4[1:]
print ('uP_assay_4')
print (uP_assay_4)

In [ ]:
#uS

uS_assay_1 = np.diff(substrate_assay_1)/substrate_assay_1[1:]
print ('uS_assay_1')
print (uS_assay_1)

uS_assay_2 = np.diff(substrate_assay_2)/substrate_assay_2[1:]
print ('uS_assay_2')
print (uS_assay_2)

uS_assay_3 = np.diff(substrate_assay_3)/substrate_assay_3[1:]
print ('uS_assay_3')
print (uS_assay_3)

uS_assay_4 = np.diff(substrate_assay_4)/substrate_assay_4[1:]
print ('uS_assay_4')
print (uS_assay_4)

In [ ]:
#We must consider that for assay_1, at t=0, uX = uxMax = 0.5649. So we need to append this value in the array
uX_1 = np.append(coef_a1_2h, uX_assay_1)
uX_2 = np.append(coef_a2_2h, uX_assay_2)
uX_3 = np.append(coef_a3_2h, uX_assay_3)
uX_4 = np.append(coef_a4_2h, uX_assay_4)
print (uX_2)

In [ ]:
#Up coefficients
uP_1 = np.append(coef_a1p_2h, uP_assay_1)
uP_2 = np.append(coef_a2p_2h, uP_assay_2)
uP_3 = np.append(coef_a3p_2h, uP_assay_3)
uP_4 = np.append(coef_a4p_2h, uP_assay_4)
print (uP_1)

In [ ]:
# Now we plot values for uX vs.time. 
#This figure can be compared with fig. 7.4 from book (pg. 141)
fig, ax1 = plt.subplots()
ax1.plot(time_assay_1, uX_1, 'ro--')
ax1.set_xlabel('time (h)')
ax1.set_ylabel('uX_1 (1/h)')
ax1.set_ylim (-0.1, .65)
plt.show()

Now, lets compute uX and P for  substrate residual values of 10 and 20 mg/L
Values presented here show a small difference compared to Table 7.4. It can be explained due to the interpolation method odopted even the accuracy of computational tools applyed.

First step is to interpolate experimental data, thus we need to import the library 'interpolate'

### Assay_1

In [ ]:
#product_1
f_product_assay_1 = interpolate.interp1d(time_assay_1, product_assay_1,kind = 'linear')
new_time_assay_1 = np.linspace(0, 7,1000)
plt.plot(time_assay_1, product_assay_1, 'o', new_time_assay_1, f_product_assay_1(new_time_assay_1), '--')
plt.legend(['product_assay_1', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_product_assay_1 = f_product_assay_1(new_time_assay_1)[36:37]
resid10g_product_assay_1 = f_product_assay_1(new_time_assay_1)[438:439]
print (resid20g_product_assay_1)
print (resid10g_product_assay_1)

In [ ]:
#substrate_1
f_substrate_assay_1 = interpolate.interp1d(time_assay_1, substrate_assay_1,kind = 'linear')
new_time_assay_1 = np.linspace(0, 7,1000)
plt.plot(time_assay_1, substrate_assay_1, 'o', new_time_assay_1, f_substrate_assay_1(new_time_assay_1), '--')
plt.legend(['substrate_assay_1', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_substrate_assay_1 = f_substrate_assay_1(new_time_assay_1)[35:36]
resid10g_substrate_assay_1 = f_substrate_assay_1(new_time_assay_1)[438:439]
print (resid20g_substrate_assay_1)
print (resid10g_substrate_assay_1)

In [ ]:
#uX_1
f_uX_1 = interpolate.interp1d(time_assay_1, uX_1,kind = 'linear')
new_time_assay_1 = np.linspace(0, 7,1000)
plt.plot(time_assay_1, uX_1, 'o', new_time_assay_1, f_uX_1(new_time_assay_1), '--')
plt.legend(['uX_1', 'linear'], loc = 'best')
plt.show()


In [ ]:
resid20g_uX_1 = f_uX_1(new_time_assay_1)[35:36]
resid10g_uX_1 = f_uX_1(new_time_assay_1)[438:439]
print (resid20g_uX_1)
print (resid10g_uX_1)

In [ ]:
#uP_1
f_uP_1 = interpolate.interp1d(time_assay_1, uP_1,kind = 'linear')
new_time_assay_1 = np.linspace(0, 7,1000)
plt.plot(time_assay_1, uX_1, 'o', new_time_assay_1, f_uP_1(new_time_assay_1), '--')
plt.legend(['uP_1', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_uP_1 = f_uP_1(new_time_assay_1)[35:36]
resid10g_uP_1 = f_uP_1(new_time_assay_1)[438:439]
print (resid20g_uP_1)
print (resid10g_uP_1)

### Assay_2

In [ ]:
#product_2
f_product_assay_2 = interpolate.interp1d(time_assay_2, product_assay_2,kind = 'linear')
new_time_assay_2 = np.linspace(0, 9,99)
plt.plot(time_assay_2, product_assay_2, 'o', new_time_assay_2, f_product_assay_2(new_time_assay_2), '--')
plt.legend(['product_assay_2', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_product_assay_2 = f_product_assay_2(new_time_assay_2)[70:71]
resid10g_product_assay_2 = f_product_assay_2(new_time_assay_2)[75:76]
print (resid20g_product_assay_2)
print (resid10g_product_assay_2)

In [ ]:
#substrate_2
f_substrate_assay_2 = interpolate.interp1d(time_assay_2, substrate_assay_2,kind = 'linear')
new_time_assay_2 = np.linspace(0, 9,99)
plt.plot(time_assay_2, substrate_assay_2, 'o', new_time_assay_2, f_substrate_assay_2(new_time_assay_2), '--')
plt.legend(['substrate_assay_2', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_substrate_assay_2 = f_substrate_assay_2(new_time_assay_2)[70:71]
resid10g_substrate_assay_2 = f_substrate_assay_2(new_time_assay_2)[75:76]
print (resid20g_substrate_assay_2)
print (resid10g_substrate_assay_2)

In [ ]:
# Replacing values by the mean in time 7hrs
#uX_2
new_uX_2 = np.average(uX_2[6:9])
#print (new_uX_2)
uX_2_ =  np.where(uX_2==uX_2[7:8],new_uX_2, uX_2)
#print (uX_2_)
f_uX_2 = interpolate.interp1d(time_assay_2, uX_2_,kind = 'linear')
new_time_assay_2 = np.linspace(0, 9,99)
plt.plot(time_assay_2, uX_2_, 'o', new_time_assay_2, f_uX_2(new_time_assay_2), '--')
plt.legend(['uX_2', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_uX_2 = f_uX_2(new_time_assay_2)[70:71]
resid10g_uX_2 = f_uX_2(new_time_assay_2)[75:76]
print (resid20g_uX_2)
print (resid10g_uX_2)

### Assay_3

In [ ]:
#product_3
f_product_assay_3 = interpolate.interp1d(time_assay_3, product_assay_3,kind = 'linear')
new_time_assay_3 = np.linspace(0, 12,560)
plt.plot(time_assay_3, product_assay_3, 'o', new_time_assay_3, f_product_assay_3(new_time_assay_3), '--')
plt.legend(['product_assay_3', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_product_assay_3 = f_product_assay_3(new_time_assay_3)[463:464]
resid10g_product_assay_3 = f_product_assay_3(new_time_assay_3)[488:489]
print (resid20g_product_assay_3)
print (resid10g_product_assay_3)

In [ ]:
#substrate_3
f_substrate_assay_3 = interpolate.interp1d(time_assay_3, substrate_assay_3,kind = 'linear')
new_time_assay_3 = np.linspace(0, 12,560)
plt.plot(time_assay_3, substrate_assay_3, 'o', new_time_assay_3, f_substrate_assay_3(new_time_assay_3), '--')
plt.legend(['substrate_assay_3', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_substrate_assay_3 = f_substrate_assay_3(new_time_assay_3)[463:464]
resid10g_substrate_assay_3 = f_substrate_assay_3(new_time_assay_3)[488:489]
print (resid20g_substrate_assay_3)
print (resid10g_substrate_assay_3)

In [ ]:
# Replacing values by the mean in time 10 and 11hrs

a_uX_3 = np.average(uX_3[9:12])
uX_3_ =  np.where(uX_3==uX_3[10:11],a_uX_3, uX_3)
b_uX_3 = np.average(uX_3[10:13])
uX_3_b =  np.where(uX_3_==uX_3_[11:12],b_uX_3, uX_3_)

f_uX_3 = interpolate.interp1d(time_assay_3, uX_3_b,kind = 'linear')
new_time_assay_3 = np.linspace(0, 12,560)
plt.plot(time_assay_3, uX_3_b, 'o', new_time_assay_3, f_uX_3(new_time_assay_3), '--')
plt.legend(['uX_3', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_uX_3 = f_uX_3(new_time_assay_3)[463:464]
resid10g_uX_3 = f_uX_3(new_time_assay_3)[488:489]
print (resid20g_uX_3)
print (resid10g_uX_3)

### Assay_4

In [ ]:
#product_4
f_product_assay_4 = interpolate.interp1d(time_assay_4, product_assay_4,kind = 'linear')
new_time_assay_4 = np.linspace(0, 13,1000)
plt.plot(time_assay_4, product_assay_4, 'o', new_time_assay_4, f_product_assay_4(new_time_assay_4), '--')
plt.legend(['product_assay_4', 'linear'], loc = 'best')
plt.show()

In [ ]:
f_product_assay_4(new_time_assay_4)
resid20g_product_assay_4 = f_product_assay_4(new_time_assay_4)[904:905]
resid10g_product_assay_4 = f_product_assay_4(new_time_assay_4)[934:935]
print (resid20g_product_assay_4)
print (resid10g_product_assay_4)

In [ ]:
#substrate_4
f_substrate_assay_4 = interpolate.interp1d(time_assay_4, substrate_assay_4,kind = 'linear')
new_time_assay_4 = np.linspace(0, 13,1000)
plt.plot(time_assay_4, substrate_assay_4, 'o', new_time_assay_4, f_substrate_assay_4(new_time_assay_4), '--')
plt.legend(['substrate_assay_4', 'linear'], loc = 'best')
plt.show()

In [ ]:
f_substrate_assay_4(new_time_assay_4)
resid20g_substrate_assay_4 = f_substrate_assay_4(new_time_assay_4)[904:905]
resid10g_substrate_assay_4 = f_substrate_assay_4(new_time_assay_4)[934:935]
print (resid20g_substrate_assay_4)
print (resid10g_substrate_assay_4)

In [ ]:
### Replacing values by the mean in time from 10 to 12 hrs
a_uX_4 = np.average(uX_4[10:14])
uX_4_ =  np.where(uX_4==uX_4[11:12],a_uX_4, uX_4)
b_uX_4 = np.average(uX_4[11:14])
uX_4_b =  np.where(uX_4_==uX_4_[12:13],b_uX_4, uX_4_)
f_uX_4 = interpolate.interp1d(time_assay_4, uX_4_b,kind = 'linear')
new_time_assay_4 = np.linspace(0, 13,1000)
plt.plot(time_assay_4, uX_4, 'o', new_time_assay_4, f_uX_4(new_time_assay_4), '--')
plt.legend(['uX_4_', 'linear'], loc = 'best')
plt.show()

In [ ]:
resid20g_uX_4 = f_uX_4(new_time_assay_4)[904:905]
resid10g_uX_4 = f_uX_4(new_time_assay_4)[934:935]
print (resid20g_uX_4)
print (resid10g_uX_4)

# Inhibition Model

In [ ]:
#First, we need to concatenate in an unique array the data previously computeted for each condition

# 1. Residual of 20 g/L

product_20mg = np.concatenate ((resid20g_product_assay_1, resid20g_product_assay_2, resid20g_product_assay_3, resid20g_product_assay_4))
print (product_20mg)
uX_20mg = np.concatenate((resid20g_uX_1, resid20g_uX_2, resid20g_uX_3, resid20g_uX_4))
#print(uX_20mg)

substrate_20mg = np.concatenate ((resid20g_substrate_assay_1, resid20g_substrate_assay_2, resid20g_substrate_assay_3, resid20g_substrate_assay_4))
print (substrate_20mg)
uX_20mg = np.concatenate((resid20g_uX_1, resid20g_uX_2, resid20g_uX_3, resid20g_uX_4))

# 2. Residual of 10 g/L

product_10mg = np.concatenate ((resid10g_product_assay_1, resid10g_product_assay_2, resid10g_product_assay_3, resid10g_product_assay_4))
print (product_10mg)
uX_10mg = np.concatenate((resid10g_uX_1, resid10g_uX_2, resid10g_uX_3, resid10g_uX_4))
#print(uX_10mg)

### Hyperbolic Inhibition

In [ ]:
##########Hyperbolic Inhibition: Residual of 10 mg/L (book result)
a = np.array([0.255, 0.161, 0.0901, 0.0364])
x1 = np.array([4.78, 21.2, 33.7, 54.7])
y1 = 1/a
slope, intercept, r_value, p_value, std_err = stats.linregress(x1, y1)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1+intercept
plt.plot(x1,line,'r-',x1,y1,'o')
plt.show()

In [ ]:
##########Hyperbolic Inhibition: Residual of 10 mg/L (estimated result)

x1 = product_10mg
y1 = 1/uX_10mg
slope, intercept, r_value, p_value, std_err = stats.linregress(x1, y1)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1+intercept
plt.plot(x1,line,'r-',x1,y1,'o')
plt.show()

In [ ]:
##########Hyperbolic Inhibition: Residual of 20 mg/L (Book Result)
a = np.array([0.565, 0.219, 0.129, 0.053])
x2 = np.array([2.12, 18.0, 30.0, 50.5])
y2 = 1/a
slope, intercept, r_value, p_value, std_err = stats.linregress(x2, y2)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x2+intercept
plt.plot(x2,line,'r-',x2,y2,'o')
plt.show()

In [ ]:
##########Hyperbolic Inhibition: Residual of 20 mg/L (Estimated Result)
x2 = product_20mg
y2 = 1/uX_20mg
slope, intercept, r_value, p_value, std_err = stats.linregress(x2, y2)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x2+intercept
plt.plot(x2,line,'r-',x2,y2,'o')
plt.show()

In [ ]:
##########Exponential Inhibition: Residual of 10 mg/L (book result)
a = np.array([0.255, 0.161, 0.0901, 0.0364])
x1_b = np.array([4.78, 21.2, 33.7, 54.7])
y1_b = np.log(a)*(-1)
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_b, y1_b)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_b+intercept
plt.plot(x1_b,line,'r-',x1_b,y1_b,'o')
plt.show()

### Exponential Inhibition

In [ ]:
##########Exponential Inhibition: Residual of 10 mg/L (estimated result)

x1_b = product_10mg
y1_b = np.log(uX_10mg)*(-1)
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_b, y1_b)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_b+intercept
plt.plot(x1_b,line,'r-',x1_b,y1_b,'o')
plt.show()

In [ ]:
##########Exponential Inhibition: Residual of 20 mg/L (Book Result)
a = np.array([0.565, 0.219, 0.129, 0.053])
x2_c = np.array([2.12, 18.0, 30.0, 50.5])
y2_c = np.log(a)*(-1)
slope, intercept, r_value, p_value, std_err = stats.linregress(x2_c, y2_c)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x2_c+intercept
plt.plot(x2_c,line,'r-',x2_c,y2_c,'o')
plt.show()

In [ ]:
##########Exponential Inhibition: Residual of 20 mg/L (Estimated Result)
x2_c = product_20mg
y2_c = np.log(uX_20mg)*(-1)
slope, intercept, r_value, p_value, std_err = stats.linregress(x2_c, y2_c)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x2_c+intercept
plt.plot(x2_c,line,'r-',x2_c,y2_c,'o')
plt.show()

### Linear Inhibition

In [ ]:
##########Linear Inhibition: Residual of 10 mg/L (book result)
x1_d = np.array([4.78, 21.2, 33.7, 54.7])
y1_d = np.array([0.255, 0.161, 0.0901, 0.0364])
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_d, y1_d)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_d+intercept
plt.plot(x1_d,line,'r-',x1_d,y1_d,'o')
plt.show()

In [ ]:
##########Linear Inhibition: Residual of 10 mg/L (estimated result)
x1_d = product_10mg
y1_d = uX_10mg
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_d, y1_d)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_d+intercept
plt.plot(x1_d,line,'r-',x1_d,y1_d,'o')
plt.show()

In [ ]:
##########Linear Inhibition: Residual of 20 mg/L (book result)
x1_e = np.array([2.12, 18.0, 30.0, 50.5])
y1_e = np.array([0.565, 0.219, 0.129, 0.053])
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_e, y1_e)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_e+intercept
plt.plot(x1_e,line,'r-',x1_e,y1_e,'o')
plt.show()

In [ ]:
##########Linear Inhibition: Residual of 20 mg/L (estimated result)
x1_e = product_20mg
y1_e = uX_20mg
slope, intercept, r_value, p_value, std_err = stats.linregress(x1_e, y1_e)
print ('r value', r_value)
print  ('p_value', p_value)
print ('standard deviation', std_err)
print ('slope', slope)
print ('intercept', intercept)
line = slope*x1_e+intercept
plt.plot(x1_e,line,'r-',x1_e,y1_e,'o')
plt.show()

                        ...this notebook is open to contribution to continue its solution!
 
                                                                      PERAZZOLI, Simone